In [ ]:
# Algorithmic Strategies Mini Project
# Course: Design and Analysis of Algorithms Lab
# Author: <yourname>

"""
This notebook implements and analyzes four real-world problems using different algorithmic strategies:
1. Scheduling TV Commercials (Greedy)
2. Maximizing Profit with Limited Budget (Dynamic Programming)
3. Sudoku Solver (Backtracking)
4. Password Cracking (Brute Force)
"""

import time, tracemalloc, random, itertools
import matplotlib.pyplot as plt
import numpy as np

from contextlib import contextmanager

@contextmanager
def measure_time_and_memory():
    tracemalloc.start()
    t0 = time.perf_counter()
    yield
    t1 = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print(f"Time taken: {t1 - t0:.6f}s | Peak memory: {peak/1024:.2f} KiB")

def schedule_ads(ads):
    """
    ads: list of tuples (id, deadline, profit)
    Returns: selected ads list and total profit
    """
    ads.sort(key=lambda x: x[2], reverse=True)
    max_deadline = max(ad[1] for ad in ads)
    slots = [None] * (max_deadline + 1)
    total_profit = 0
    selected = []

    for ad in ads:
        ad_id, deadline, profit = ad
        for d in range(deadline, 0, -1):
            if slots[d] is None:
                slots[d] = ad_id
                total_profit += profit
                selected.append(ad)
                break
    return selected, total_profit


# Example run
ads = [('A', 2, 100), ('B', 1, 19), ('C', 2, 27), ('D', 1, 25), ('E', 3, 15)]
with measure_time_and_memory():
    chosen, total = schedule_ads(ads)
print("Selected ads:", chosen)
print("Total profit:", total)

def generate_ads(n, max_deadline=50):
    return [(f"Ad{i}", random.randint(1, max_deadline), random.randint(10, 200)) for i in range(n)]

sizes = [100, 300, 600, 1000]
times, mems = [], []

for n in sizes:
    ads = generate_ads(n)
    tracemalloc.start()
    t0 = time.perf_counter()
    schedule_ads(ads)
    t1 = time.perf_counter()
    _, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    times.append(t1 - t0)
    mems.append(peak / 1024)

plt.plot(sizes, times, marker='o')
plt.title("TV Commercial Scheduling: Input size vs Time")
plt.xlabel("Number of Ads")
plt.ylabel("Time (s)")
plt.grid(True)
plt.show()

plt.plot(sizes, mems, marker='o', color='red')
plt.title("TV Commercial Scheduling: Input size vs Memory (KiB)")
plt.xlabel("Number of Ads")
plt.ylabel("Memory (KiB)")
plt.grid(True)
plt.show()


def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        for w in range(capacity + 1):
            if weights[i - 1] <= w:
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - weights[i - 1]] + values[i - 1])
            else:
                dp[i][w] = dp[i - 1][w]

    res = dp[n][capacity]
    return res


values = [60, 100, 120]
weights = [10, 20, 30]
capacity = 50
with measure_time_and_memory():
    print("Max profit:", knapsack(values, weights, capacity))


sizes = [10, 30, 60, 100]
times, mems = [], []

for n in sizes:
    vals = [random.randint(10, 100) for _ in range(n)]
    wts = [random.randint(5, 15) for _ in range(n)]
    cap = int(sum(wts) * 0.4)
    tracemalloc.start()
    t0 = time.perf_counter()
    knapsack(vals, wts, cap)
    t1 = time.perf_counter()
    _, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    times.append(t1 - t0)
    mems.append(peak / 1024)

plt.plot(sizes, times, marker='o')
plt.title("Knapsack: Items vs Time")
plt.xlabel("Number of Items")
plt.ylabel("Time (s)")
plt.grid(True)
plt.show()

plt.plot(sizes, mems, marker='o', color='red')
plt.title("Knapsack: Items vs Memory (KiB)")
plt.xlabel("Number of Items")
plt.ylabel("Memory (KiB)")
plt.grid(True)
plt.show()


def is_valid(board, r, c, num):
    for i in range(9):
        if board[r][i] == num or board[i][c] == num:
            return False
    start_r, start_c = 3 * (r // 3), 3 * (c // 3)
    for i in range(start_r, start_r + 3):
        for j in range(start_c, start_c + 3):
            if board[i][j] == num:
                return False
    return True

def solve_sudoku(board):
    for r in range(9):
        for c in range(9):
            if board[r][c] == 0:
                for num in range(1, 10):
                    if is_valid(board, r, c, num):
                        board[r][c] = num
                        if solve_sudoku(board):
                            return True
                        board[r][c] = 0
                return False
    return True

# Example puzzle
puzzle = [
 [5,3,0,0,7,0,0,0,0],
 [6,0,0,1,9,5,0,0,0],
 [0,9,8,0,0,0,0,6,0],
 [8,0,0,0,6,0,0,0,3],
 [4,0,0,8,0,3,0,0,1],
 [7,0,0,0,2,0,0,0,6],
 [0,6,0,0,0,0,2,8,0],
 [0,0,0,4,1,9,0,0,5],
 [0,0,0,0,8,0,0,7,9]
]

with measure_time_and_memory():
    solve_sudoku(puzzle)
print(np.matrix(puzzle))




def brute_force_crack(target, charset):
    for l in range(1, len(target) + 1):
        for attempt in itertools.product(charset, repeat=l):
            if ''.join(attempt) == target:
                return ''.join(attempt), l

target = "ab"
charset = "abc"

with measure_time_and_memory():
    found, length = brute_force_crack(target, charset)
print("Password found:", found)


lengths = [2, 3, 4]
charset = "abc123"
times = []

for l in lengths:
    pwd = charset[:l]
    t0 = time.perf_counter()
    brute_force_crack(pwd, charset)
    t1 = time.perf_counter()
    times.append(t1 - t0)

plt.plot(lengths, times, marker='o', color='purple')
plt.title("Password Cracking: Password Length vs Time")
plt.xlabel("Password Length")
plt.ylabel("Time (s)")
plt.grid(True)
plt.show()




import pandas as pd

summary = pd.DataFrame({
    "Problem": ["TV Commercial Scheduling", "Knapsack", "Sudoku Solver", "Password Cracking"],
    "Strategy": ["Greedy", "Dynamic Programming", "Backtracking", "Brute Force"],
    "Domain": ["Media & Advertisement", "Budget Planning", "Gaming", "Cybersecurity"],
    "Complexity (Approx.)": ["O(n log n)", "O(nW)", "Exponential", "O(|charset|^n)"]
})
summary